In [ ]:
from convertors.convert_to_json import *
import pandas as pd
import os

In [ ]:
global keys_dict
Key_dataframe = pd.read_csv(os.path.abspath(r"C:\Users\Axel\Documents\PYTHON\MSP_V3\FragHub\TOOLS\Generate_csv_from_original_files\datas\key_to_convert.csv"),sep=";", encoding="UTF-8") # Remplacez 'your_file.csv' par le chemin de votre fichier
keys_dict = dict(zip(Key_dataframe['known_synonym'], Key_dataframe['fraghub_default'].str.upper()))

global keys_list
keys_list = ['FILENAME',
             'PREDICTED',
             'FRAGHUBID',
             'SPECTRUMID',
             'RESOLUTION',
             'SYNON',
             'CHARGE',
             'IONIZATION',
             'MSLEVEL',
             'FRAGMENTATIONMODE',
             'NAME',
             'PRECURSORMZ',
             'EXACTMASS',
             'AVERAGEMASS',
             'PRECURSORTYPE',
             'INSTRUMENTTYPE',
             'INSTRUMENT',
             'SMILES',
             'INCHI',
             'INCHIKEY',
             'COLLISIONENERGY',
             'FORMULA',
             'RETENTIONTIME',
             'IONMODE',
             'COMMENT',
             'NUM PEAKS',
             'PEAKS_LIST']

In [ ]:
def convert_keys(dict_list):
    """
    Convert keys in metadata_dict based on the provided keys_dict and keys_list.

    :param metadata_dict: A dictionary containing metadata information.
    :return: A dictionary with converted keys based on the provided keys_dict and keys_list.
    """
    output = []
    for metadata_dict in dict_list:
        converted = {keys_dict[key.lower()]: val for key, val in metadata_dict.items() if key.lower() in keys_dict and keys_dict[key.lower()] in keys_list}
    
        converted.update({key: "" for key in keys_list if key not in converted})
        output.append(converted)

    return output

In [ ]:
original_db_path = r"C:\Users\Axel\Documents\MSP_DB\ORIGINALS_msp_DB\DB Janvier 2024\DB_publi"

In [ ]:
FINAL_MSP, FINAL_XML, FINAL_CSV, FINAL_JSON, FINAL_MGF = convert_to_json(original_db_path)

In [ ]:
FINAL_MSP = convert_keys(FINAL_MSP)

In [ ]:
FINAL_XML = convert_keys(FINAL_XML)

In [ ]:
FINAL_CSV = convert_keys(FINAL_CSV)

In [ ]:
FINAL_JSON = convert_keys(FINAL_JSON)

In [ ]:
FINAL_MGF = convert_keys(FINAL_MGF)

In [ ]:
msp_df = pd.DataFrame(FINAL_MSP)
del FINAL_MSP

In [ ]:
xml_df = pd.DataFrame(FINAL_XML)
del FINAL_XML

In [ ]:
csv_df = pd.DataFrame(FINAL_CSV)
del FINAL_CSV

In [ ]:
json_df = pd.DataFrame(FINAL_JSON)
del FINAL_JSON

In [ ]:
mgf_df = pd.DataFrame(FINAL_MGF)
del FINAL_MGF

In [ ]:
msp_df.to_csv(r"C:\Users\Axel\Documents\PYTHON\MSP_V3\FragHub\TOOLS\Generate_csv_from_original_files\output\msp_df.csv", sep=";", quotechar='"', index=False)

In [ ]:
xml_df.to_csv(r"C:\Users\Axel\Documents\PYTHON\MSP_V3\FragHub\TOOLS\Generate_csv_from_original_files\output\xml_df.csv", sep=";", quotechar='"', index=False)

In [ ]:
csv_df.to_csv(r"C:\Users\Axel\Documents\PYTHON\MSP_V3\FragHub\TOOLS\Generate_csv_from_original_files\output\csv_df.csv", sep=";", quotechar='"', index=False)

In [ ]:
json_df.to_csv(r"C:\Users\Axel\Documents\PYTHON\MSP_V3\FragHub\TOOLS\Generate_csv_from_original_files\output\json_df.csv", sep=";", quotechar='"', index=False)

In [ ]:
mgf_df.to_csv(r"C:\Users\Axel\Documents\PYTHON\MSP_V3\FragHub\TOOLS\Generate_csv_from_original_files\output\mgf_df.csv", sep=";", quotechar='"', index=False)